In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot

In [33]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

In [34]:
# loading data from parquet file
import os
file_path = os.path.join(os.getcwd(), 'yellow_tripdata_2024-05.parquet')

df = pd.read_parquet(file_path)


In [35]:
df.sample(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
1250495,2,2024-05-12 00:19:11,2024-05-12 00:28:14,3.0,1.42,1.0,N,79,186,1,10.0,1.0,0.5,3.00,0.0,1.0,18.00,2.5,0.0
3228605,1,2024-05-31 09:49:55,2024-05-31 10:01:05,1.0,1.20,1.0,N,113,68,1,10.0,2.5,0.5,2.80,0.0,1.0,16.80,2.5,0.0
3038465,2,2024-05-29 15:34:38,2024-05-29 15:54:34,1.0,2.30,1.0,N,238,262,1,19.1,0.0,0.5,4.62,0.0,1.0,27.72,2.5,0.0
1529058,2,2024-05-14 18:33:53,2024-05-14 18:56:15,1.0,2.91,1.0,N,246,211,1,21.9,2.5,0.5,5.68,0.0,1.0,34.08,2.5,0.0
1458387,1,2024-05-14 08:53:54,2024-05-14 09:08:47,1.0,1.30,1.0,N,234,233,1,12.8,2.5,0.5,3.35,0.0,1.0,20.15,2.5,0.0
1850988,2,2024-05-17 13:56:40,2024-05-17 14:12:20,1.0,2.49,1.0,N,75,42,1,17.0,0.0,0.5,0.80,0.0,1.0,19.30,0.0,0.0
2357658,2,2024-05-22 08:28:03,2024-05-22 08:34:43,1.0,1.22,1.0,N,229,237,1,8.6,0.0,0.5,1.00,0.0,1.0,13.60,2.5,0.0
1643477,2,2024-05-15 18:26:30,2024-05-15 18:52:20,1.0,0.00,1.0,N,107,237,1,20.5,2.5,0.5,5.40,0.0,1.0,32.40,2.5,0.0
1137206,1,2024-05-11 00:36:41,2024-05-11 00:45:22,1.0,1.90,1.0,N,163,50,1,11.4,3.5,0.5,3.25,0.0,1.0,19.65,2.5,0.0
1277230,1,2024-05-12 11:32:29,2024-05-12 11:35:33,0.0,0.90,1.0,N,166,152,1,6.5,0.0,0.5,1.60,0.0,1.0,9.60,0.0,0.0


In [36]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 3723833 entries, 0 to 3723832
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     str           
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee            float64   

# Checking the null values 

In [37]:
na_count = df.isnull().sum()
na_pct = (
    df
    .isnull()
    .sum()
    .div(df.shape[0])
    .mul(100)
    .round(2)
    
)

(
    pd
    .concat([na_count, na_pct], axis=1)
    .set_axis(['Count', '%'], axis=1)
    .query('Count > 0')
    .sort_values('Count', ascending=False)
)

,Count,%
passenger_count,404666,10.87
RatecodeID,404666,10.87
store_and_fwd_flag,404666,10.87
congestion_surcharge,404666,10.87
Airport_fee,404666,10.87


## found around 10% of the data has null values

# Checking duplicate entires 

In [38]:
df.duplicated().sum()

np.int64(0)